In [1]:
using DataFrames
using Clustering

In [2]:
using CSV
using ProgressMeter

In [3]:
function elbow(d)
    data = reshape(coalesce.(d.mjd, 0), (1, size(d.mjd)[1]))
    kms = [kmeans(data, i) for i in 2:5]
    costs = [km.totalcost for km in kms]
    diff1 = costs[1] - costs[2]
    diff2 = costs[2] - costs[3]
    diff3 = costs[3] - costs[4]
    if diff1 / diff2 > diff2 / diff3
        return kms[2].assignments
    else
        return kms[3].assignments
    end
end
    

elbow (generic function with 1 method)

In [4]:
function add_cluster(df)
    clustered = by(df, :object_id, elbow)
    df[:cluster] = clustered[:, :x1]
    df
end

add_cluster (generic function with 1 method)

In [5]:
headrow = 2
limit = 50000000
test = CSV.File("/home/hidehisa/.kaggle/competitions/plasticc/test_set.csv"; header=1, skipto=headrow, limit=limit) |> DataFrame
while size(test)[1] > 0
    last_id = unique(test.object_id)[end]
    len_last = size(test[test.object_id .== last_id, :])[1]
    test = test[1:limit-len_last, :]
    clustered = add_cluster(test)
    endrow = headrow + limit-len_last
    CSV.write("test_with_cluster_$(headrow)_$(endrow).csv", clustered)
    headrow = endrow + 1
    test = CSV.File("/home/hidehisa/.kaggle/competitions/plasticc/test_set.csv"; header=1, skipto=headrow, limit=limit) |> DataFrame
end

┌ Warning: indexing with colon as row will create a copy in the future use df[col_inds] to get the columns without copying
│   caller = add_cluster(::DataFrame) at In[4]:3
└ @ Main ./In[4]:3


ArgumentError: ArgumentError: unable to check bounds for indices of type Missing

In [6]:
headrow

449999412

In [7]:
test = CSV.File("/home/hidehisa/.kaggle/competitions/plasticc/test_set.csv"; header=1, skipto=headrow) |> DataFrame
last_id = unique(test.object_id)[end]

104853812

In [8]:
size(test)

(3653694, 6)

In [9]:
clustered = add_cluster(test)
CSV.write("test_with_cluster_$(headrow)_end.csv", clustered)

"test_with_cluster_449999412_end.csv"